In [422]:
import os, subprocess
import xml.sax
import mwparserfromhell 
import json
import re
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import sys
import os
from multiprocessing import Pool 
import tqdm 
from itertools import chain
from functools import partial
import gc
import json

In [464]:
petscan = pd.read_csv('data/math/math_petscan_1216.csv') 

petscan['title_clean'] = petscan['title'].apply(lambda x: x.replace('_', ' '))

petscan_title = petscan['title_clean']

sparql = pd.read_csv('data/math/math_clean_sparql_786', sep='\t')

sparql_title = sparql['article_name']

input_table = pd.read_csv('data/math/total_math_11697', sep='\t') 

input_titles = input_table['title_clean'].tolist()

input_titles = pd.concat([sparql_title, petscan_title], axis=0)

len(input_titles)

input_titles.drop_duplicates(inplace=True)

input_titles = input_titles.tolist()

len(input_titles), type(input_titles)

In [465]:
dumps = os.listdir('wiki_dump/')
os.stat('wiki_dump/' + dumps[4]).st_size / 1e6

313.261382

In [466]:
data_path = 'wiki_dump/' + dumps[4]

In [477]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._articles = []
        self._total_articles = 0
        self._articles_count = 0

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._total_articles += 1
            # Search through the page to see if the page is a target_article
            article = process_article(**self._values)
            # Append to the list of articles
            if article:
                self._articles_count += 1
                self._articles.append(article)

In [478]:
def process_article(title, text, timestamp):
    """Process a wikipedia article looking for template"""
    
    # Create a parsing object
    wiki = mwparserfromhell.parse(text)
    
    if title in input_titles:
        # check if title in list of target articles

        # Extract the simple, stripped_code text
        striped_code = wiki.strip_code().strip()
        
        # Extract headings, if they exist
        headings = wiki.filter_headings()

        # Extract internal wikilinks, avoid "Category:", "File:"
        wikilinks = []
        for i in wiki.filter_wikilinks():
            i = i.strip('[]')
            if 'File:' not in i and \
            'Category:' not in i and \
            'Wikipedia:' not in i and \
            'en:' not in i:
                link_list.append(i)
                
        # return articles title, striped_text, headings, wikilinks
        return (title, striped_code, headings, wikilinks)

In [479]:
start = timer()
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# Parse the entire file
for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    if (i + 1) % 10000 == 0:
        sys.stdout.write('\r' +f'Processed {i + 1} lines so far.')
    try:
        parser.feed(line)
    except StopIteration:
        break
    
end = timer()

print(f'\nSearched through {handler._article_count} articles.')
print(f'\nFound {len(articles)} articles in {round(end - start)} seconds.')

Processed 22260000 lines so far.

AttributeError: 'WikiXmlHandler' object has no attribute '_article_count'

In [470]:
articles = handler._articles

In [472]:
handler._total_articles

33

In [475]:
articles[0][1]

'360px|thumbnail \n  \n The theory of ologs is an attempt to provide a rigorous mathematical framework for knowledge representation, construction of scientific models and data storage using category theory, linguistic and graphical tools. Ologs were introduced in 2010 by David Spivak, < ref name=Spivak >  < /ref >  a research scientist in the Department of Mathematics, MIT. \n \n ==Etymology== \n The term  " olog "  is short for  " ontology log " .  " Ontology "  derives from onto-, from the Greek ὤν, ὄντος  " being; that which is " , present participle of the verb εἰμί  " be " , and -λογία, -logia: science, study, theory. \n \n ==Mathematical formalism== \n At the basic level an olog  < math > \\mathcal{C} < /math >  is a category whose objects are represented as boxes containing sentences and whose morphisms are represented as directed labeled arrows between boxes. The structures of the sentences for both the objects and the morphisms of  < math > \\mathcal{C} < /math >  need to be c